In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualizations_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display

In [12]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
import time
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [4]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf")
box_file = FindResourceOrThrow("drake/C_Iris_Examples/assets/shelves.sdf")


parser = Parser(plant)


models = []
models.append(Parser(plant, scene_graph).AddModelFromFile(model_file))
models.append(Parser(plant, scene_graph).AddModelFromFile(box_file))

sp = 0.4
x_fac = 1.2
locs = [ [0,0,0], 
        [x_fac*sp, 1.4*sp,0.4], [x_fac*sp,-1.4*sp,0.4], [-x_fac*sp,-1.4*sp,0.4], [-x_fac*sp,1.4*sp,0.4], 
        [0.0 ,0 , 0.95], [0.0 ,0 , -0.05]] 
idx = 0
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(),
                     RigidTransform(locs[idx]))
    idx +=1


plant.Finalize()


        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros_like(plant.GetPositionLowerLimits())

#compute limits in t-space
limits_t = []
for q in [plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), q_star))

do_viz = True

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram

# This line will run marching cubes to generate a mesh of the C-space obstacle
# Increase N to increase the resolution of the C-space obstacle.


INFO:drake:Meshcat listening for connections at http://localhost:7002
INFO:drake:Meshcat listening for connections at http://localhost:7003
/home/amice/Documents/coding_projects/drake/C_Iris_Examples/iris_plant_visualizer.py:31: DrakeDeprecationWarning: MeshcatVisualizerCpp has been renamed to MeshcatVisualizer. The deprecated code will be removed from Drake on or after 2022-11-01.
  self.vis = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, self.meshcat1)


## Set up the sliders so we can move the plant around manually

In [5]:
sliders = []
for i in range(plant.num_positions()):
    sliders.append(widgets.FloatSlider(min=plant.GetPositionLowerLimits()[i],
                                       max=plant.GetPositionUpperLimits()[i], 
                                       value=0, description=f'q{i}'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = np.zeros(plant.num_positions())
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.showres(q)
    visualizer.visualize_planes()
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)




FloatSlider(value=0.0, description='q0', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='q1', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='q2', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='q3', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='q4', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='q5', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='q6', max=3.05433, min=-3.05433)

In [6]:
# filter fused joints self collisions so they don't interfere with collision engine
digaram = visualizer.diagram
context = visualizer.diagram_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()

# gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
# get_name_of_gid = lambda gid : inspector.GetName(gid)
# gids.sort(key=get_name_of_gid)
# iiwa_oneDOF_gids = [gid for gid in gids if "iiwa7_oneDOF::" in get_name_of_gid(gid)]
# iiwa_twoDOF_gids = [gid for gid in gids if "iiwa7_twoDOF::" in get_name_of_gid(gid)]

# oneDOF_fused_col_geom = iiwa_oneDOF_gids[2:]
# iiwa_oneDOF_fused_set = GeometrySet(oneDOF_fused_col_geom)
# twoDOF_fused_col_geom = iiwa_twoDOF_gids[4:]
# iiwa_twoDOF_fused_set = GeometrySet(twoDOF_fused_col_geom)
# scene_graph.collision_filter_manager()\
#             .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_oneDOF_fused_set))
# scene_graph.collision_filter_manager()\
#             .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_twoDOF_fused_set))

# Setup IRIS Options and Generate Regions

In [7]:
# Some examples of some seed points which give large regions using the
# non-linear programming approach to IRIS described in Algorithm 3
seed_points_q = np.zeros(plant.num_positions())[:, np.newaxis]
seed_points = np.array([Ratfk.ComputeTValue(seed_points_q[idx], q_star)\
                        for idx in range(seed_points_q.shape[0])])



In [8]:
regions = []
ellipses = []

iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 20
iris_options.configuration_space_margin = 1e-5
iris_options.max_faces_per_collision_pair = 60
iris_options.termination_threshold = -1
iris_options.q_star = q_star
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False

for i, s in enumerate(seed_points):
    print(s.shape)
    print(q_star.shape)
    q = Ratfk.ComputeQValue(s, iris_options.q_star)
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), q)
    r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
    regions.append(r)
    ellipses.append(r.MaximumVolumeInscribedEllipsoid())


(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)


In [9]:
regions[0].A().shape

(87, 7)

# Certify one region

In [10]:
cspace_free_region_certifier = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)
filtered_collision_pairs = set()
solver_options = mp.SolverOptions()
# make the solver verbose
solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 1)

## first search for a minimal, uniform contraction of a proposed region which can be certified

In [13]:
i = 0
region_to_certify = regions[i]
seed_point = seed_points[i,:]
if do_viz:
    visualizer.plot_regions([region_to_certify], ellipses=None, region_suffix='_original_to_certify')

binary_search_options = rational_forward_kinematics.BinarySearchOption()
binary_search_options.epsilon_max = 0 # it is very unlikely that we can find a uniform expansion of the current region
binary_search_options.max_iters = 4
# speed up the bisection search by taking non-uniform steps when possible
binary_search_options.search_d = True
# find the smallest e such that At <= b + e1 still contains our seed point.
binary_search_options.epsilon_min = rational_forward_kinematics.FindEpsilonLower(region_to_certify.A(), 
                                                                                 region_to_certify.b(),
                                                                                 limits_t[0], limits_t[1],
                                                                                 seed_point)
#use as many threads as possible to speed up computation
binary_search_options.num_threads = -1

t0 = time.time()
certified_region_contraction_solution = cspace_free_region_certifier.CspacePolytopeBinarySearch(q_star,
                                                                 filtered_collision_pairs,
                                                                 region_to_certify.A(),
                                                                 region_to_certify.b(),
                                                                 binary_search_options, 
                                                                 solver_options,
                                                                 seed_point)
t1 = time.time()
print(f"Time to binary search = {t1-t0}")
certified_region_contraction = HPolyhedron(certified_region_contraction_solution.C,
                                           certified_region_contraction_solution.d)


(iiwa7::iiwa_link_0_collision, iiwa7::iiwa_link_3_collision)
(iiwa7::iiwa_link_0_collision, iiwa7::iiwa_link_4_collision)
(iiwa7::iiwa_link_0_collision, iiwa7::iiwa_link_5_collision)
(iiwa7::iiwa_link_0_collision, iiwa7::iiwa_link_6_collision)
(iiwa7::iiwa_link_0_collision, iiwa7::iiwa_link_7_collision)
(iiwa7::iiwa_link_1_collision, iiwa7::iiwa_link_3_collision)
(iiwa7::iiwa_link_2_collision, iiwa7::iiwa_link_5_collision)
(iiwa7::iiwa_link_2_collision, iiwa7::iiwa_link_7_collision)
(iiwa7::iiwa_link_2_collision, shelves::shelf_lower)
(iiwa7::iiwa_link_2_collision, shelves::shelf_upper)
(iiwa7::iiwa_link_3_collision, iiwa7::iiwa_link_5_collision)
(iiwa7::iiwa_link_3_collision, shelves::top)
(iiwa7::iiwa_link_3_collision, shelves::shelf_lower)
(iiwa7::iiwa_link_3_collision, shelves::shelf_upper)
(iiwa7::iiwa_link_4_collision, shelves::top)
(iiwa7::iiwa_link_4_collision, shelves::shelf_lower)
(iiwa7::iiwa_link_4_collision, shelves::shelf_upper)
(iiwa7::iiwa_link_5_collision, iiwa7::iiwa_

RuntimeError: binary search: the initial epsilon -0.034555243428750555 is infeasible

## Now that we have a certified region, we can grow it using bilinear alternations

In [16]:
bilinear_alternation_options = rational_forward_kinematics.BilinearAlternationOption()
bilinear_alternation_options.max_iters = 20
bilinear_alternation_options.lagrangian_backoff_scale = 0
bilinear_alternation_options.polytope_backoff_scale = 0
# number of threads used to solve this optimization.
bilinear_alternation_options.num_threads = -1

A = np.vstack([np.eye(plant.num_positions()),
               - np.eye(plant.num_positions())])
b = 1e-3*np.ones(A.shape[0])
certified_region_final_solution, polytope_volumes, ellipsoid_determinants = cspace_free_region_certifier.CspacePolytopeBilinearAlternation(q_star,
                                                                 filtered_collision_pairs,
                                                                 A,
                                                                 # it is helpful to pullback the solution of the previous step a little bit
                                                                 b,
                                                                 bilinear_alternation_options, 
                                                                 solver_options,
                                                                 seed_point)
final_certified_region = HPolyhedron(certified_region_final_solution.C, certified_region_final_solution.d)
if do_viz:
    visualizer.plot_regions([final_certified_region], ellipses=None,
                            region_suffix='_certified_region_final')

INFO:drake:Found Lagrangian multiplier and separating planes
INFO:drake:Lagrangian step time 9.143 s
INFO:drake:max(power(det(P), 1/8))=0.002371372850701074, solver_time 0.05985307693481445
INFO:drake:cost improvement inf
INFO:drake:mosek info 5, InfeasibleConstraints


## Visualizing the separating planes
Our certificate of non-collision for a given region contains one parametric, separating hyperplane per collision pair. In this scene, there are 137 pairs of objects which could collide. We plot two planes of interest. After running this cell, you can move the plant with the slider and see how the planes move as function of the configuration.

In [ ]:
oneDOF_iiwa_end_gid = iiwa_oneDOF_gids[-1]
twoDOF_iiwa_end_gid = iiwa_twoDOF_gids[-1]
twoDOF_iiwa_base_gid = iiwa_twoDOF_gids[0]
id_pairs_of_interest = [(oneDOF_iiwa_end_gid, twoDOF_iiwa_end_gid),
                        (oneDOF_iiwa_end_gid, twoDOF_iiwa_base_gid)]

            
visualizer.collision_pairs_of_interest = id_pairs_of_interest
visualizer.certified_region_solution_list = [certified_region_final_solution]